In [2]:
# This code was written in TF 1.12 but should be supported all the way through
# TF 1.15. Untested in TF 2.0+.
%tensorflow_version 1.x

# Download the raw data (only 108 epoch data points, for full dataset,
# uncomment the second line for nasbench_full.tfrecord).

!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
# !curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

# Clone and install the code and dependencies.

!git clone https://github.com/google-research/nasbench
!pip install ./nasbench

# Initialize the NASBench object which parses the raw data into memory (this
# should only be run once as it takes up to a few minutes).
from nasbench import api

import sys
if './nasbench/nasbench/' not in sys.path:
  sys.path.append('./nasbench/nasbench/')

from api import *

dataset_file = 'nasbench_only108.tfrecord'

# Use nasbench_full.tfrecord for full dataset (run download command above).
nasbench = api.NASBench(dataset_file)

TensorFlow 1.x selected.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498M  100  498M    0     0  88.4M      0  0:00:05  0:00:05 --:--:-- 91.0M
Cloning into 'nasbench'...
remote: Enumerating objects: 96, done.
remote: Total 96 (delta 0), reused 0 (delta 0), pack-reused 96
Unpacking objects: 100% (96/96), done.
Processing ./nasbench
  Created wheel for nasbench: filename=nasbench-1.0-cp36-none-any.whl size=46788 sha256=9d11c05ec934d03ffe698b7ac0245a4fdfaaed9097e107b7597f6c9fc1b9e8d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-v05x6p90/wheels/4b/19/99/1d5fdfe30f8b16fab91e900808f4f7e5adc38e602c84970ad5
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=9336d2e2df75a2106786c6502796b2e8ea7ab623397110c50f923f47b26f8573
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully bui

In [3]:
# Standard imports
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

In [5]:
count = 0
max_count = 6

for serialized_row in tf.python_io.tf_record_iterator(dataset_file):
  # Parse the data from the data file.
  module_hash, epochs, raw_adjacency, raw_operations, raw_metrics = (
      json.loads(serialized_row.decode('utf-8')))

  dim = int(np.sqrt(len(raw_adjacency)))
  adjacency = np.array([int(e) for e in list(raw_adjacency)], dtype=np.int8)
  adjacency = np.reshape(adjacency, (dim, dim))
  operations = raw_operations.split(',')
  metrics = model_metrics_pb2.ModelMetrics.FromString(
      base64.b64decode(raw_metrics))

  # Evaluation statistics at the end of training
  final_evaluation = metrics.evaluation_data[2]
  training_time = final_evaluation.training_time
  train_accuracy = final_evaluation.train_accuracy
  validation_accuracy = (
      final_evaluation.validation_accuracy)
  test_accuracy = final_evaluation.test_accuracy
  trainable_params = metrics.trainable_parameters

  print(f'Module {count//3+1}.{count%3+1} \nAdjacency matrix: \n{adjacency} \nOperations: {operations} \nTrainable parameters: {trainable_params}')

  print(f'Train Accuracy: {train_accuracy} \nValidation Accuracy: {validation_accuracy} \nTest Accuracy: {test_accuracy}\n')

  count += 1
  if count == max_count:
    break

Module 1.1 
Adjacency matrix: 
[[0 1 0 0 1 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]] 
Operations: ['input', 'conv3x3-bn-relu', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'output'] 
Trainable parameters: 8555530
Train Accuracy: 1.0 
Validation Accuracy: 0.9241786599159241 
Test Accuracy: 0.9211738705635071

Module 1.2 
Adjacency matrix: 
[[0 1 0 0 1 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]] 
Operations: ['input', 'conv3x3-bn-relu', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'output'] 
Trainable parameters: 8555530
Train Accuracy: 1.0 
Validation Accuracy: 0.9245793223381042 
Test Accuracy: 0.9190705418586731

Module 1.3 
Adjacency matrix: 
[[0 1 0 0 1 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]] 
Operations: ['input', 'conv3x3-bn-relu', 'ma